In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,7"
import warnings 
warnings.filterwarnings('ignore') 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed
set_seed(42)

MODEL_NAME = "HuggingFaceH4/zephyr-7b-alpha"
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True, load_in_8bit=False, 
                                             device_map="auto", cache_dir="/home/scratch-buffalo/hjin008/huggingface_llm")
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) 
tokenizer.sep_token = tokenizer.unk_token
tokenizer.cls_token = tokenizer.unk_token
tokenizer.mask_token = tokenizer.unk_token

# pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16, device_map="auto",)


def text2json_trans(txt):
    # messages = [{"role": "system","content": "Your are a smart junior programmer."},
    #             {"role": "user", "content": f'''Please summarize the following text related personal information as a json which only includes name, age, gender, and BMI: {txt}'''}]
    # prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # # print(outputs[0]["generated_text"])
    # # print('--'*60)
    # text = outputs[0]["generated_text"]
    # start = text.find("{")
    # end = text.find("}", start) + 1
    # json_text = text[start:end]
    # json_data = json.loads(json_text)
    # # print(json_text)
    # return json_data
    import json
    import re
    messages = [{"role": "system","content": "Your are a smart junior programmer."},
                {"role": "user", "content": f'''Please summarize the following text related personal information as a json.
                the json which only includes name, age, BMI, GenHlth(general health, scale 1-5 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor), 
                HighBP(0 = no, 1 = yes), DiffWalk(0 = no, 1 = yes), HighChol(0 = no, 1 = yes), and HeartDiseaseorAttack(0 = no, 1 = yes): {txt}'''}]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True)
    # print(outputs[0]["generated_text"])
    # print('--'*60)
    text = outputs[0]["generated_text"]
    start = text.find("{")
    end = text.find("}", start) + 1
    json_text = text[start:end]
    json_text = re.sub(r'(?<!\\")//.*', '', json_text)
    json_data = json.loads(json_text)
    # print(json_text)
    return json_data


def generate_response(user_input):
    # add the user's reponse to the chat records
    messages.append({"role": "user", "content": user_input})

    # transfer messages list by the chat template for llm better understanding and generating
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # llm generating response
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    # get the llm response
    generated_text = outputs[0]["generated_text"]
    assistant_response = generated_text[len(prompt):].strip()

    # add the AI assistant's response to the chat records
    messages.append({"role": "assistant", "content": assistant_response})

    return assistant_response


# remove some non-literal part to make the chat records clean
def post_process(text):
    for icon in ['<|>','\n', "<|assistant|>"]:
        text = text.replace(icon, "").strip()
    return text

[2023-11-12 19:26:05,503] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
# system information and initail chat recode
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a professional nurse.",
    },
    {
        "role": "system", 
        "content": '''The following is a conversation between a assistant an user. As a hospital nurse, the assistant is engaging in a conversation with the user to inquire about some information including user's name, age, BMI, GenHlth(general health), HighBP, DiffWalk, HighChol, and HeartDiseaseorAttack. 
The conversation should be conducted in a dialogue format, with each question asked individually. The interaction should be characterized by a friendly and patient demeanor. 
The conversation will begin with a greeting from assistant, followed by the questions. Rememble ask one question a time.
After all the questions have been asked, assistant will summarize the information gathered and confirm its accuracy with user. Ok, take a deep breath, and let's go!'''},
]

# runing the chat function
while True:
    user_input = input("user: ")
    if "that is right." in user_input.lower(): break
    response = generate_response(user_input)
    response = post_process(response)
    print("assistant:", response)

# change the summary information to JSON
res = text2json_trans(messages[-1]['content'].replace('<|assistant|>',' ').replace('\n',' '))
res

user:  Hi


assistant: Hello! How may I assist you today? Please allow me to introduce myself. My name is NurseBot, and I'm here to help you with any health-related questions you may have. What is your name?


user:  peter jack


assistant: Hello, Peter Jack. Can you please tell me your age?


user:  21


...

assistant: Thank you for providing me with all of this information, Peter Jack. Can you please confirm that the information I have is correct? Is there anything else you would like to add or clarify?


user:  could you summarize them for me to confirm?


assistant: Certainly, Peter Jack. Let me summarize the information you provided for you:- Name: Peter Jack- Age: 21- BMI: 24- GenHlth: Good, no ongoing medical conditions- HighBP: No- DiffWalk: No- HighChol: No- HeartDiseaseorAttack: NoIs this information correct? Please let me know if you have any other details you would like to add or clarify.


user:  that is right.


{'name': 'Peter Jack', 'age': 21, 'gender': None, 'bmi': 24}

In [32]:
txt = messages[-1]['content'].replace('<|assistant|>',' ').replace('\n',' ')
txt

"Please summarize the following text related personal information as a json.             the json which only includes name, age, BMI, GenHlth(general health, scale 1-5 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor),              HighBP(0 = no, 1 = yes), DiffWalk(0 = no, 1 = yes), HighChol(0 = no, 1 = yes), and HeartDiseaseorAttack(0 = no, 1 = yes):         <|> Certainly! Based on the information you've provided, your name is Peter Jack, you're a 9-year-old female, your height is 170 centimeters, your weight is 49 kilograms, your BMI is [BMI calculated], you're a vegetarian, your annual income is 30,000, and you don't have any medical conditions or recent surgeries or procedures to report. Is that correct?"

In [30]:
import json
import re
messages = [{"role": "system","content": "Your are a smart junior programmer."},
            {"role": "user", "content": f'''Please summarize the following text related personal information as a json.
            the json which only includes name, age, BMI, GenHlth(general health, scale 1-5 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor), 
            HighBP(0 = no, 1 = yes), DiffWalk(0 = no, 1 = yes), HighChol(0 = no, 1 = yes), and HeartDiseaseorAttack(0 = no, 1 = yes): {txt}'''}]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True)
print(outputs[0]["generated_text"])
print('--'*60)
text = outputs[0]["generated_text"]
start = text.find("{")
end = text.find("}", start) + 1
json_text = text[start:end]
json_text = re.sub(r'(?<!\\")//.*', '', json_text)
json_data = json.loads(json_text)
print(json_text)

<|system|>
Your are a smart junior programmer.</s>
<|user|>
Please summarize the following text related personal information as a json.
            the json which only includes name, age, BMI, GenHlth(general health, scale 1-5 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor), 
            HighBP(0 = no, 1 = yes), DiffWalk(0 = no, 1 = yes), HighChol(0 = no, 1 = yes), and HeartDiseaseorAttack(0 = no, 1 = yes):         <|> Certainly! Based on the information you've provided, your name is Peter Jack, you're a 9-year-old female, your height is 170 centimeters, your weight is 49 kilograms, your BMI is [BMI calculated], you're a vegetarian, your annual income is 30,000, and you don't have any medical conditions or recent surgeries or procedures to report. Is that correct?</s>
<|assistant|>
No, I was mistaken. The given text in the previous message was actually an error on my part. I apologize. The information I was provided with is not related to personal information of a person. The t

In [31]:
json_data

{'name': 'John Doe',
 'age': 35,
 'bmi': 24.5,
 'genHlth': 4,
 'highBP': 1,
 'diffWalk': 0,
 'highChol': 1,
 'heartDiseaseorAttack': 0}